In [2]:
#%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import numpy as np
import sys

plt.rcParams['savefig.facecolor'] = "0.8"

In [4]:
def example_plot(ax, fontsize=12):
    ax.plot([1, 2])

    ax.locator_params(nbins=3)
    ax.set_xlabel('x-label', fontsize=fontsize)
    ax.set_ylabel('y-label', fontsize=fontsize)
    ax.set_title('Title', fontsize=fontsize)

In [5]:
from matplotlib.widgets import AxesWidget
import six

class VertSlider(AxesWidget):
    """
    A slider representing a floating point range.

    For the slider to remain responsive you must maintain a
    reference to it.

    The following attributes are defined
      *ax*        : the slider :class:`matplotlib.axes.Axes` instance

      *val*       : the current slider value

      *hline*     : a :class:`matplotlib.lines.Line2D` instance
                     representing the initial value of the slider

      *poly*      : A :class:`matplotlib.patches.Polygon` instance
                     which is the slider knob

      *valfmt*    : the format string for formatting the slider text

      *label*     : a :class:`matplotlib.text.Text` instance
                     for the slider label

      *closedmin* : whether the slider is closed on the minimum

      *closedmax* : whether the slider is closed on the maximum

      *slidermin* : another slider - if not *None*, this slider must be
                     greater than *slidermin*

      *slidermax* : another slider - if not *None*, this slider must be
                     less than *slidermax*

      *dragging*  : allow for mouse dragging on slider

    Call :meth:`on_changed` to connect to the slider event
    """
    def __init__(self, ax, label, valmin, valmax, valinit=0.5, valfmt='%1.2f',
                 closedmin=True, closedmax=True, slidermin=None,
                 slidermax=None, dragging=True, **kwargs):
        """
        Create a slider from *valmin* to *valmax* in axes *ax*.

        Additional kwargs are passed on to ``self.poly`` which is the
        :class:`matplotlib.patches.Rectangle` which draws the slider
        knob.  See the :class:`matplotlib.patches.Rectangle` documentation
        valid property names (e.g., *facecolor*, *edgecolor*, *alpha*, ...).

        Parameters
        ----------
        ax : Axes
            The Axes to put the slider in

        label : str
            Slider label

        valmin : float
            The minimum value of the slider

        valmax : float
            The maximum value of the slider

        valinit : float
            The slider initial position

        label : str
            The slider label

        valfmt : str
            Used to format the slider value, fprint format string

        closedmin : bool
            Indicate whether the slider interval is closed on the bottom

        closedmax : bool
            Indicate whether the slider interval is closed on the top

        slidermin : Slider or None
            Do not allow the current slider to have a value less than
            `slidermin`

        slidermax : Slider or None
            Do not allow the current slider to have a value greater than
            `slidermax`


        dragging : bool
            if the slider can be dragged by the mouse

        """
        AxesWidget.__init__(self, ax)

        self.valmin = valmin
        self.valmax = valmax
        self.val = valinit
        self.valinit = valinit
        self.poly = ax.axhspan(valmin, valinit, 0, 1, **kwargs)

        self.hline = ax.axhline(valinit, 0, 1, color='r', lw=1)

        self.valfmt = valfmt
        ax.set_xticks([])
        ax.set_ylim((valmin, valmax))
        ax.set_yticks([])
        ax.set_navigate(False)

        self.connect_event('button_press_event', self._update)
        self.connect_event('button_release_event', self._update)
        if dragging:
            self.connect_event('motion_notify_event', self._update)
        self.label = ax.text(0.5, 1.03, label, transform=ax.transAxes,
                             verticalalignment='center',
                             horizontalalignment='center')

        self.valtext = ax.text(0.5, -0.03, valfmt % valinit,
                               transform=ax.transAxes,
                               verticalalignment='center',
                               horizontalalignment='center')

        self.cnt = 0
        self.observers = {}

        self.closedmin = closedmin
        self.closedmax = closedmax
        self.slidermin = slidermin
        self.slidermax = slidermax
        self.drag_active = False

    def _update(self, event):
        """update the slider position"""
        if self.ignore(event):
            return

        if event.button != 1:
            return

        if event.name == 'button_press_event' and event.inaxes == self.ax:
            self.drag_active = True
            event.canvas.grab_mouse(self.ax)

        if not self.drag_active:
            return

        elif ((event.name == 'button_release_event') or
              (event.name == 'button_press_event' and
               event.inaxes != self.ax)):
            self.drag_active = False
            event.canvas.release_mouse(self.ax)
            return

        val = event.ydata
        if val <= self.valmin:
            if not self.closedmin:
                return
            val = self.valmin
        elif val >= self.valmax:
            if not self.closedmax:
                return
            val = self.valmax

        if self.slidermin is not None and val <= self.slidermin.val:
            if not self.closedmin:
                return
            val = self.slidermin.val

        if self.slidermax is not None and val >= self.slidermax.val:
            if not self.closedmax:
                return
            val = self.slidermax.val

        self.set_val(val)

    def set_val(self, val):
        xy = self.poly.xy
        xy[1] = 0, val
        xy[2] = 1, val
        self.poly.xy = xy
        self.valtext.set_text(self.valfmt % val)
        if self.drawon:
            self.ax.figure.canvas.draw_idle()
        self.val = val
        if not self.eventson:
            return
        for cid, func in six.iteritems(self.observers):
            func(val)

    def on_changed(self, func):
        """
        When the slider value is changed, call *func* with the new
        slider position

        A connection id is returned which can be used to disconnect
        """
        cid = self.cnt
        self.observers[cid] = func
        self.cnt += 1
        return cid

    def disconnect(self, cid):
        """remove the observer with connection id *cid*"""
        try:
            del self.observers[cid]
        except KeyError:
            pass

    def reset(self):
        """reset the slider to the initial value if needed"""
        if (self.val != self.valinit):
            self.set_val(self.valinit)



In [6]:
## keypress demo
keylist = []
def press(event):
    print('press', event.key)
    keylist.append(event.key)
    sys.stdout.flush()
    if event.key == 'x':
        visible = xl.get_visible()
        xl.set_visible(not visible)
        fig.canvas.draw()
    
def handle_close(evt):
    print('Closed Figure!')
    print(keylist)


# Fixing random state for reproducibility
np.random.seed(19680801)

fig, ax = plt.subplots()

fig.canvas.mpl_connect('key_press_event', press)
fig.canvas.mpl_connect('close_event', handle_close)

ax.plot(np.random.rand(12), np.random.rand(12), 'go')
xl = ax.set_xlabel('easy come, easy go')
ax.set_title('Press a key')
plt.show()

Closed Figure!
[]


In [105]:
##  a zoom in window
##  mutlicursor
## And checkboxes
## and radio buttons
from matplotlib.widgets import MultiCursor
from matplotlib.widgets import CheckButtons
from matplotlib.widgets import RadioButtons
from matplotlib.widgets import Slider, Button
from scipy.signal import medfilt

# from utility_funcs import VertSlider
wavelength_halfwidth = 5
minwave = 0
maxwave = 30

axcolor = 'lightgoldenrodyellow'

smooth_noise_dict = {'Original': tog1, 'Smooth': tog2}
default_dict = { 'default':(0,1,1.),
                 'predicted from default':(0.05,1.2,1.),
                 'predicted from history':(0.1,1.15,1.),
                 'from history':(0.18,1.18,1.)    
                }

steps = (0.01,0.01,0.01)
default_key = 'default'
spectra_is_good = True
t = np.arange(minwave, maxwave, 0.01)
s0 = np.sin(0.125*np.pi*t)
s1 = np.sin(0.25*np.pi*t)
s2 = np.sin(0.5*np.pi*t)
s3 = np.sin(1*np.pi*t)

def pix_to_wave(xs,coefs):
    return coefs['a'] + coefs['b']*xs + coefs['c']*np.power(xs,2) + \
            coefs['d']*np.power(xs,3) + coefs['e']*np.power(xs,4) + \
            coefs['f']*np.power(xs,5)

coefs = {}
coefs['a'],coefs['b'],coefs['c'] = default_dict[default_key]
coefs['d'],coefs['e'],coefs['f'] = 0,0,0
pixels = np.arange(len(t))
waves = pix_to_wave(pixels,coefs)
tog1 = np.sin(0.25*np.pi*t)
tog2 = medfilt(tog1,3)

meanwave = (maxwave+minwave)//2
fig = plt.figure()
axsrc = plt.subplot2grid((1, 3), (0, 0), colspan=2)
axzoom = plt.subplot2grid((1, 3), (0, 2))
#fig, (axsrc, axzoom) = plt.subplots(nrows=1,ncols=2)

## standard 
slines, zlines = [],[]
labels = ('Hg','Ne','Ar','Xe')
colors = ('k','r','g','c')
visibility = (False,False,False,True)
for linex,liney,label,color,visible in zip([t,t,t,t],[s0,s1,s2,s3],labels,colors,visibility):
    ## signal
    sl, = axsrc.plot(linex, liney, visible=visible, lw=2, color=color, label=label)
    slines.append(sl)
    ## zoomed
    zl, = axzoom.plot(linex, liney, visible=visible, lw=2, color=color, label=label)
    zlines.append(zl)
    
## Signal
stogl, = axsrc.plot(waves, tog1, lw=2, color='b', label='Original')
ztogl, = axzoom.plot(waves, tog1, lw=2, color='b', label='Original')

## Button and slider funcs
def showunshow_lines(label):
    index = labels.index(label)
    slines[index].set_visible(not slines[index].get_visible())
    zlines[index].set_visible(not zlines[index].get_visible())
    plt.draw()

def zoom_adjust(event):
    if event.button != 1:
        return
    elif (event.inaxes==axsrc or event.inaxes==axzoom):
        x, y = event.xdata, event.ydata
        axzoom.set_xlim(x - wavelength_halfwidth, x + wavelength_halfwidth)
        fig.canvas.draw()

def smooth_noise_flip(label):
    ydata = smooth_noise_dict[label]
    stogl.set_ydata(ydata)
    ztogl.set_ydata(ydata)
    plt.draw()

def change_default_sliderset(label):
    #samp.valinit,sfreq.valinit,squad.valinit = default_dict[label] 
    samp.valinit,sfreq.valinit,throwaway = default_dict[label] 
    del throwaway
    samp.vline.set_xdata([default_dict[label][0]]*len(samp.vline.get_xdata()))
    sfreq.vline.set_xdata([default_dict[label][1]]*len(sfreq.vline.get_xdata()))
    #squad.vline.set_xdata([default_dict[label][2]]*len(sfreq.vline.get_xdata()))
    plt.draw()  
    
def slider_spec_update(val):
    coefs['a'] = samp.val+samp_fine.val
    coefs['b'] = sfreq.val+sfreq_fine.val
    coefs['c'] = squad_fine.val#squad.val+squad_fine.val
    waves = pix_to_wave(pixels,coefs)
    stogl.set_xdata(waves)
    ztogl.set_xdata(waves)
    fig.canvas.draw_idle()

def reset_sliders(event):
    sfreq.reset()
    samp.reset()
    #squad.reset()
    sfreq_fine.reset()
    samp_fine.reset()
    squad_fine.reset()
    
def flag_spec(event):
    print("Spec flagged as bad")
    spectra_is_good = False
    plt.close()
 
def save_and_close(event):
    print("Spec closed with the following params: a={} b-{} c={}".format(coefs['a'],coefs['b'],\
                                                                        coefs['c']))
    plt.close()

def print_to_screen(event):
    print("a={} b-{} c={}".format(coefs['a'],coefs['b'],coefs['c']))    
    
## Make checkbuttons with all plotted lines with correct visibility
## [x,y,width,height]
plot_ystart = 0.36

slider_ystart = 0.03

slider_xstart = 0.04
boxes_xstart_row1 = 0.7
boxes_xstart_row2 = 0.8
boxes_xstart_row3 = 0.92

box_width = 0.08
box_width2 = 0.14
slider_width = 0.62

height2 = 0.1
height3 = 0.15
height_slider = 0.03
height_button = 0.04

## Move subplot over to make room for checkboxes
plt.subplots_adjust(left=slider_xstart,right=1-slider_xstart,\
                    bottom=plot_ystart,top=1-slider_xstart)

## Change the name and limits of each axis
axsrc.set(xlim=(minwave,maxwave), ylim=(-1.1, 1.1), autoscale_on=False,
          title='Click to zoom')
axzoom.set(xlim=(meanwave-wavelength_halfwidth, meanwave+wavelength_halfwidth), \
           ylim=(-1.1, 1.1), autoscale_on=False, title='Zoom window')

## Setup button locations
#slider1_rax = plt.axes([slider_xstart, slider_ystart+10*height_slider, slider_width, height_slider], facecolor=axcolor)
slider2_rax = plt.axes([slider_xstart, slider_ystart+8*height_slider, slider_width, height_slider], facecolor=axcolor)
slider3_rax = plt.axes([slider_xstart, slider_ystart+6*height_slider, slider_width, height_slider], facecolor=axcolor)
slider4_rax = plt.axes([slider_xstart, slider_ystart+4*height_slider, slider_width, height_slider], facecolor=axcolor)
slider5_rax = plt.axes([slider_xstart, slider_ystart+2*height_slider, slider_width, height_slider], facecolor=axcolor)
slider6_rax = plt.axes([slider_xstart, slider_ystart, slider_width, height_slider], facecolor=axcolor)

linelists_rax = plt.axes([boxes_xstart_row1, slider_ystart, box_width, height3], facecolor=axcolor)

spec_radio_rax = plt.axes([boxes_xstart_row1, 0.20, box_width, height2], facecolor=axcolor)
def_radio_rax = plt.axes([boxes_xstart_row2, 0.15, box_width2, height3], facecolor=axcolor)

close_rax = plt.axes([boxes_xstart_row3, slider_ystart,      0.05, height_button])
reset_rax = plt.axes([boxes_xstart_row3, slider_ystart+0.06, 0.05, height_button])
flag_rax =  plt.axes([boxes_xstart_row2, slider_ystart,      0.1, height_button])
print_rax = plt.axes([boxes_xstart_row2, slider_ystart+0.06, 0.1, height_button])

## Checkboxes
linelist_check = CheckButtons(linelists_rax, labels, visibility)
## Radio boxes
spec_radio = RadioButtons(spec_radio_rax, ('Original','Smooth'))
def_radio = RadioButtons(def_radio_rax, default_dict.keys())
## Sliders
samp = Slider(slider2_rax, 'offset', -100., 100.0, valinit=default_dict[default_key][0], valstep=steps[0])
sfreq = Slider(slider3_rax, 'stretch', 0.1, 30.0, valinit=default_dict[default_key][1], valstep=steps[1])
#squad = Slider(slider3_rax, 'quad', -10.0, 10.0, valinit=default_dict[default_key][2], valstep=steps[2])
samp_fine = Slider(slider4_rax, 'fine offset', -0.5, 0.5, valinit=0, valstep=steps[0]/100)
sfreq_fine = Slider(slider5_rax, 'fine stretch', -0.5, 0.5, valinit=0, valstep=steps[1]/100)
squad_fine = Slider(slider6_rax, 'fine quad', -0.5, 0.5, valinit=0, valstep=steps[2]/100)
## Buttons
reset_button = Button(reset_rax, 'Reset', color=axcolor, hovercolor='0.975')
flag_button = Button(flag_rax, 'Flag as Bad', color=axcolor, hovercolor='0.975')
close_button = Button(close_rax, 'Close', color=axcolor, hovercolor='0.975')
print_button = Button(print_rax, 'Print to Terminal', color=axcolor, hovercolor='0.975')


## Run the interactive buttons
fig.canvas.mpl_connect('button_press_event', zoom_adjust)
linelist_check.on_clicked(showunshow_lines)
spec_radio.on_clicked(smooth_noise_flip)
def_radio.on_clicked(change_default_sliderset)

sfreq.on_changed(slider_spec_update)
samp.on_changed(slider_spec_update)
#squad.on_changed(slider_spec_update)
sfreq_fine.on_changed(slider_spec_update)
samp_fine.on_changed(slider_spec_update)
squad_fine.on_changed(slider_spec_update)

reset_button.on_clicked(reset_sliders)
flag_button.on_clicked(flag_spec)
close_button.on_clicked(save_and_close)
print_button.on_clicked(print_to_screen)
multi = MultiCursor(fig.canvas, (axsrc, axzoom), color='r', lw=1)

## plot the final canvas in a pop-up window
figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()
plt.show()

Spec closed with the following params: a=0.1 b-1.15 c=0.0


In [38]:
zlines

In [41]:
%matplotlib qt5
"""
===========
Slider Demo
===========

Using the slider widget to control visual properties of your plot.

In this example, a slider is used to choose the frequency of a sine
wave. You can control many continuously-varying properties of your plot in
this way.
"""
from matplotlib.widgets import Slider, Button, RadioButtons

fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.25)
t = np.arange(0.0, 1.0, 0.001)
a0 = 5
f0 = 3
delta_f = 5.0
s = a0*np.sin(2*np.pi*f0*t)
l, = plt.plot(t, s, lw=2, color='red')
plt.axis([0, 1, -10, 10])

axcolor = 'lightgoldenrodyellow'
axfreq = plt.axes([0.25, 0.1, 0.65, 0.03], facecolor=axcolor)
axamp = plt.axes([0.25, 0.15, 0.65, 0.03], facecolor=axcolor)

sfreq = Slider(axfreq, 'Freq', 0.1, 30.0, valinit=f0, valstep=delta_f)
samp = Slider(axamp, 'Amp', 0.1, 10.0, valinit=a0)


def update(val):
    amp = samp.val
    freq = sfreq.val
    l.set_ydata(amp*np.sin(2*np.pi*freq*t))
    fig.canvas.draw_idle()
sfreq.on_changed(update)
samp.on_changed(update)

resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')


def reset(event):
    sfreq.reset()
    samp.reset()
button.on_clicked(reset)

rax = plt.axes([0.025, 0.5, 0.15, 0.15], facecolor=axcolor)
radio = RadioButtons(rax, ('red', 'blue', 'green'), active=0)


def colorfunc(label):
    l.set_color(label)
    fig.canvas.draw_idle()
radio.on_clicked(colorfunc)

plt.show()

In [28]:
##  rectangule selector
from matplotlib.widgets import RectangleSelector

def line_select_callback(eclick, erelease):
    'eclick and erelease are the press and release events'
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata
    print("(%3.2f, %3.2f) --> (%3.2f, %3.2f)" % (x1, y1, x2, y2))
    print(" The button you used were: %s %s" % (eclick.button, erelease.button))


def toggle_selector(event):
    print(' Key pressed.')
    if event.key in ['Q', 'q'] and toggle_selector.RS.active:
        print(' RectangleSelector deactivated.')
        toggle_selector.RS.set_active(False)
    if event.key in ['A', 'a'] and not toggle_selector.RS.active:
        print(' RectangleSelector activated.')
        toggle_selector.RS.set_active(True)


fig, current_ax = plt.subplots()                 # make a new plotting range
N = 100000                                       # If N is large one can see
x = np.linspace(0.0, 10.0, N)                    # improvement by use blitting!

plt.plot(x, +np.sin(.2*np.pi*x), lw=3.5, c='b', alpha=.7)  # plot something
plt.plot(x, +np.cos(.2*np.pi*x), lw=3.5, c='r', alpha=.5)
plt.plot(x, -np.sin(.2*np.pi*x), lw=3.5, c='g', alpha=.3)

print("\n      click  -->  release")

# drawtype is 'box' or 'line' or 'none'
toggle_selector.RS = RectangleSelector(current_ax, line_select_callback,
                                       drawtype='box', useblit=True,
                                       button=[1, 3],  # don't use middle button
                                       minspanx=5, minspany=5,
                                       spancoords='pixels',
                                       interactive=True)
plt.connect('key_press_event', toggle_selector)
plt.show()


      click  -->  release
(2.00, -1.05) --> (6.51, 1.06)
 The button you used were: 1 1
(2.00, -1.05) --> (6.51, 1.06)
 The button you used were: 3 3
(2.00, -1.05) --> (6.51, 1.06)
 The button you used were: 1 1
(2.00, -1.05) --> (6.51, 1.06)
 The button you used were: 1 1
(2.00, -1.05) --> (6.51, 1.06)
 The button you used were: 1 1
(2.00, -1.05) --> (6.51, 1.06)
 The button you used were: 3 3
(2.00, -1.05) --> (6.51, 1.06)
 The button you used were: 3 3
(2.00, -1.05) --> (6.51, 1.06)
 The button you used were: 1 1
(2.00, -1.05) --> (6.51, 1.06)
 The button you used were: 3 3
(2.00, -1.05) --> (6.51, 1.06)
 The button you used were: 3 3
(2.00, -1.05) --> (6.51, 1.06)
 The button you used were: 3 3
(6.03, -0.26) --> (9.24, 0.59)
 The button you used were: 1 1
(0.43, -1.09) --> (4.57, 1.03)
 The button you used were: 1 1


In [30]:
##   input line for interactive plotting
from matplotlib.widgets import TextBox
fig, ax = plt.subplots()
plt.subplots_adjust(bottom=0.2)
t = np.arange(-2.0, 2.0, 0.001)
s = t ** 2
initial_text = "t ** 2"
l, = plt.plot(t, s, lw=2)


def submit(text):
    ydata = eval(text)
    l.set_ydata(ydata)
    ax.set_ylim(np.min(ydata), np.max(ydata))
    plt.draw()

axbox = plt.axes([0.1, 0.05, 0.8, 0.075])
text_box = TextBox(axbox, 'Evaluate', initial=initial_text)
text_box.on_submit(submit)

plt.show()

In [33]:
from ipywidgets import widgets, interactive
import pandas as pd
import seaborn as sns # Optional, will only affect the color of bars and the grid

In [34]:
from io import StringIO

testdata=StringIO("""Year,Sex,Area,Count
2015,W,Dhaka,6
2015,M,Dhaka,3
2015,W,Khulna,1
2015,M,Khulna,8
2014,M,Dhaka,13
2014,W,Dhaka,20
2014,M,Khulna,9
2014,W,Khulna,6
2013,W,Dhaka,11
2013,M,Dhaka,2
2013,W,Khulna,8
2013,M,Khulna,5
2012,M,Dhaka,12
2012,W,Dhaka,4
2012,W,Khulna,7
2012,M,Khulna,1
    """)

df = pd.read_csv(testdata, sep=",")

In [35]:
# Create two bounded text box that allow only numbers between the min year (2012) and the max year (2015)
start_year = widgets.BoundedFloatText(
    value=df.Year.min(),
    min=df.Year.min(),
    max=df.Year.max(),
    step=1,
    description='Start Year:',
    disabled=False,
    color='black'
)
end_year = widgets.BoundedFloatText(
    value=df.Year.max(),
    min=df.Year.min(),
    max=df.Year.max(),
    step=1,
    description='End Year:',
    disabled=False,
    color='black'
)

# Make a dropdown to select the Area, or "All"
area = widgets.Dropdown(
    options=['All'] + list(df['Area'].unique()),
    value='All',
    description='Area:',
)

def plotit(area, start_year, end_year):
    """
    Filters and plot the dataframe as a stacked bar chart of count of Male versus Women

    Args:
    -----
        * area (str): the area to filter on, or "All" to display all Areas

        * start_year, end_year (int, as float): the start and ends years, inclusive

        Note: the dataframe to plot is globally defined here as `df`

    Returns:
    --------
        A matplotlib stacked bar chart

    """
    if start_year > end_year:
        print("You must select a start year that is prior to end year")
    else:
        df2 = df.copy()
        if area != 'All':
            df2 = df2[df2.Area == area]

        # Filter between min and max years (inclusive)
        df2 = df2[(df2.Year >= start_year) & (df2.Year <= end_year)]


        # Plot it (only if there's data to plot)
        if len(df2) > 0:
            df2.groupby(['Year', 'Sex']).sum()['Count'].unstack().plot(kind='bar', stacked=True, title="Area = {}".format(area))
            plt.show();
        else:
            print("No data to show for current selection")

In [37]:
%matplotlib inline
interactive(plotit, area=area, start_year=start_year, end_year=end_year)

interactive(children=(Dropdown(description='Area:', index=1, options=('All', 'Dhaka', 'Khulna'), value='Dhaka'…